In [ ]:
import sys
import calendar
from datetime import datetime, date, time, timedelta
from datetime import datetime
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import StreamingLinearRegressionWithSGD


In [ ]:
sc = SparkContext(appName="PythonLogisticRegressionWithLBFGSExample")
ssc = StreamingContext(sc, 4)


In [ ]:
################ ARCHIVOS DESDE FILE SYSTEM DE WINDOWS ###############

ruta_data_training = 'file:///C://Users//Usuario//TATR-2020//Streaming//aa.txt'
ruta_data_test = 'file:///C://Users//Usuario//TATR-2020//Streaming//aa.txt'

In [ ]:
############## ARCHIVOS DESDE HADOOP FILE SYSTEM ###################

#ruta_data_training = '/obligatorio/b.txt'
#ruta_data_test = '/obligatorio/a.txt'

In [ ]:
def splitData(lp):
        vec= []
        vec = (lp.split('\t'))
        return vec

In [ ]:
def parseData(vec):
       
        resultado = []
        fecha = datetime.strptime(vec[0], '%Y-%m-%d')
        intFecha = int(fecha.strftime('%Y%m%d'))
        resultado.append(intFecha)
        vhora = vec[1].split(' ')
        hora = datetime.strptime(vhora[1], '%H:%M:%S')
        intHora = int(hora.strftime('%H%M%S'))
        resultado.append(intHora)
                
        for i in range(2,9):
            resultado.append(float(vec[i]))
        return resultado
    

In [ ]:
def RDDisNumecir(rdd):
    resultado = []
    try:
        resultado = []
        fecha = datetime.strptime(rdd[0], '%d/%m/%Y')
        intFecha = int(fecha.strftime('%d%m'))
        resultado.append(intFecha)
        hora = datetime.strptime(rdd[1], '%H:%M:%S')
        intHora = int(hora.strftime('%H%M%S'))
        resultado.append(intHora)
        for i in range(2,9):
                resultado.append(float(rdd[i]))
        return resultado
    except:
        logDato(rdd)
        error = []
        log = []
        for i in range(0,9):
            error.append('?')
        
        return error

In [ ]:
def labeledData(vec):
    
        label = vec[8]
        
        resultado =[]
        for i in range(0,8):
            resultado.append(float(vec[i]))
        features = Vectors.dense(resultado)
        return LabeledPoint(label, features)
                                               
                                                  

In [ ]:
trainingData = sc.textFile(ruta_data_training)

In [ ]:
testingData = sc.textFile(ruta_data_test)

In [ ]:
split_data = trainingData.map(splitData)
split_test_data = testingData.map(splitData)

In [ ]:
split_data.collect()

In [ ]:
split_test_data.collect()

In [ ]:
#################### DATA CLEANING ###################################################################

In [ ]:
ctr_filtro_1 = split_data.filter(lambda x: x[2] != '?') \
                     .filter(lambda x: x[3] != '?') \
                     .filter(lambda x: x[4] != '?') \
                     .filter(lambda x: x[5] != '?') \
                     .filter(lambda x: x[6] != '?') \
                     .filter(lambda x: x[7] != '?') \
                     .filter(lambda x: x[8] != '?')

In [ ]:
ctr_filtro_2 = ctr_filtro_1.filter(lambda x: x[2] != ' ') \
                       .filter(lambda x: x[3] != ' ') \
                       .filter(lambda x: x[4] != ' ') \
                       .filter(lambda x: x[5] != ' ') \
                       .filter(lambda x: x[6] != ' ') \
                       .filter(lambda x: x[7] != ' ') \
                       .filter(lambda x: x[8] != ' ')

In [ ]:
ctr_filtro_3 = ctr_filtro_2.filter(lambda x: x[2] != '') \
                       .filter(lambda x: x[3] != '') \
                       .filter(lambda x: x[4] != '') \
                       .filter(lambda x: x[5] != '') \
                       .filter(lambda x: x[6] != '') \
                       .filter(lambda x: x[7] != '') \
                       .filter(lambda x: x[8] != '')

In [ ]:
ctr_filtro_test_1 = split_test_data.filter(lambda x: x[2] != '?') \
                     .filter(lambda x: x[3] != '?') \
                     .filter(lambda x: x[4] != '?') \
                     .filter(lambda x: x[5] != '?') \
                     .filter(lambda x: x[6] != '?') \
                     .filter(lambda x: x[7] != '?') \
                     .filter(lambda x: x[8] != '?')

In [ ]:
ctr_filtro_test_2 = ctr_filtro_test_1.filter(lambda x: x[2] != ' ') \
                       .filter(lambda x: x[3] != ' ') \
                       .filter(lambda x: x[4] != ' ') \
                       .filter(lambda x: x[5] != ' ') \
                       .filter(lambda x: x[6] != ' ') \
                       .filter(lambda x: x[7] != ' ') \
                       .filter(lambda x: x[8] != ' ')

In [ ]:
ctr_filtro_test_3 = ctr_filtro_test_2.filter(lambda x: x[2] != '') \
                       .filter(lambda x: x[3] != '') \
                       .filter(lambda x: x[4] != '') \
                       .filter(lambda x: x[5] != '') \
                       .filter(lambda x: x[6] != '') \
                       .filter(lambda x: x[7] != '') \
                       .filter(lambda x: x[8] != '')

In [ ]:
parseo_data_test = ctr_filtro_test_3.map(parseData)

In [ ]:
parseo_data = ctr_filtro_3.map(parseData)

In [ ]:
parseo_data.collect()

In [ ]:
parseo_data_test.collect()

In [ ]:
#################### PROCESAMIENTO DE STREAMING / ALERTAS ###################################

In [ ]:
def alerta_voltaje(rdd):
    
    alerta = rdd.filter(lambda x: x[4] > 240) 
    if not alerta.isEmpty():
        print("------------------------ALERTA: EXCESO DE VOLTAJE ------------------------")
        print(alerta)
        print ('Registro: {0}'.format(alerta.collect()))
    else:
        print("------------------------VOLTAJE SIN ALERTAS-------------------------------")


In [ ]:
def alerta_intensidad(rdd):
    
    alerta = rdd.filter(lambda x: x[5] > 50) 
    if not alerta.isEmpty():
        print("------------------------ALERTA: EXCESO DE INTENSIDAD ------------------------")
        print(alerta)
        print ('Registro: {0}'.format(alerta.collect()))
    else:
        print("------------------------INTENSIDAD SIN ALERTAS-------------------------------")


In [ ]:
def alerta_ExcesoConsumoSub1(rdd):
    
    alerta = rdd.filter(lambda x: x[6] > 80) 
    if not alerta.isEmpty():
        print("------------------------ALERTA: EXCESO DE CONSUMO SUB-1 ------------------------")
        print(alerta)
        print ('Registro: {0}'.format(alerta.collect()))
    else:
        print("------------------------EXCESO DE CONSUMO SUB-1 SIN ALERTAS-------------------------------")


In [ ]:
################### ENTRENAMIENTO DE MODELO ##########################################################

In [ ]:
trainingData = parseo_data.map(labeledData)

In [ ]:
testingData = parseo_data_test.map(labeledData)

In [ ]:
trainingData.collect()

In [ ]:
testingData.collect()

In [ ]:
trainingQueue = [trainingData]  
testingQueue = [testingData]
ctrlQueue = [parseo_data]

In [ ]:
trainingStream = ssc.queueStream(trainingQueue)
testingStream = ssc.queueStream(testingQueue)
ctrlQueue = ssc.queueStream(ctrlQueue)

In [ ]:
numFeatures = 8
model = StreamingLinearRegressionWithSGD()
model.setInitialWeights([0.2, 0.1, 0.5, 0.1, 0.5, 0.6, 0.3, 0.1])

In [ ]:
model.trainOn(trainingStream)


In [ ]:
result_training = model.predictOnValues(trainingStream.map(lambda lp: (lp.label, lp.features)))
result_training.pprint()

In [ ]:
result_testing = model.predictOnValues(testingStream.map(lambda lp: (lp.label, lp.features)))
result_testing.pprint()

In [ ]:
###### GUARDAR PREDICCION EN HADOOP FILE SYSTEMA ##################################

#def toCSVLine(data):
#     return ','.join(str(d) for d in data)
#
#lineCSV = result.map(toCSVLine)
########ESTA LINEA NO FUNCIONA #######  lineCSV.saveAsTextFile('hdfs://my-node:9000/tmp/labels-and-predictions.csv')
#saved_data = lineCSV.foreachRDD(lambda row: row.saveAsTextFile("/obligatorio/resultado.csv"))

In [ ]:
####### METRICA ##############################################333

#MSE = result \
#    .map(lambda vb: (vb[0] - vb[1])**2) \
#    .reduce(lambda x, y:x+y)
#MSE.pprint()

In [ ]:
alerta1 = ctrlQueue.foreachRDD(lambda row: alerta_voltaje(row))
alerta2 = ctrlQueue.foreachRDD(lambda row: alerta_intensidad(row))
alerta3 = ctrlQueue.foreachRDD(lambda row: alerta_ExcesoConsumoSub1(row))

In [ ]:
ssc.start()
ssc.awaitTermination()